# Manejo de valores nulos y atípicos

In [1]:
import sys
import os
import pandas as pd

current_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.abspath(os.path.join(current_dir, '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from scripts.eda_analyzer import EDAAnalyzer
from scripts.transform.imputer_handler import ImputerHandler

In [2]:
df = pd.read_csv('../../data/raw/Estudiantes_clean.csv')
eda = EDAAnalyzer()
imputer = ImputerHandler()

In [3]:
eda.plot_null_heatmap(df)

Eliminar la columna 'demuestra_confianza' porque tiene muy pocos valores llenos

In [4]:
df = df.drop(columns=['demuestra_confianza'])

## Analizar variables categóricas

In [5]:
cols_categoricas = ['estrato', 'tipo_vivienda', 'interés_estudios_superiores', 'familia', 'actividades_extracurriculares', 'proyección_vocacional']

### Estrato

In [6]:
eda.plot_single_barplot(df, 'estrato')

In [7]:
df_imputed = imputer.impute_missing_values(df, 'estrato', metodo="moda")

### Tipo de vivienda

In [8]:
eda.plot_single_barplot(df, 'tipo_vivienda')

In [9]:
df_imputed = imputer.impute_missing_values(df_imputed, 'tipo_vivienda', metodo="moda")

### Interés en estudios superiores

In [10]:
eda.plot_single_barplot(df, 'interés_estudios_superiores')

In [11]:
df_imputed = imputer.impute_missing_values(df_imputed, 'interés_estudios_superiores', metodo="moda")

### Composición familiar

In [12]:
eda.plot_single_barplot(df, 'familia')

In [13]:
df_imputed = imputer.impute_missing_values(df_imputed, 'familia', metodo="moda")

### Actividades extracurriculares

In [14]:
eda.plot_single_barplot(df, 'actividades_extracurriculares')

In [15]:
df_imputed = imputer.impute_missing_values(df_imputed, 'actividades_extracurriculares', metodo="moda")

### Proyección vocacional

In [16]:
eda.plot_single_barplot(df, 'proyección_vocacional')

In [17]:
df_imputed = imputer.impute_missing_values(df_imputed, 'proyección_vocacional', metodo="moda")

## Analizar variables numéricas

In [18]:
variables_numericas = ['horas_semana_estudio_casa', 'total_hermanos']

In [19]:
# Estadísticas descriptivas
df_imputed[variables_numericas].describe()


,horas_semana_estudio_casa,total_hermanos
count,281.000000,281.000000
mean,2.084386,1.231317
std,5.375950,1.085562
min,0.000000,0.000000
25%,0.000000,1.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,68.000000,8.000000


In [20]:
eda.plot_histograms(df_imputed, variables_numericas)

In [21]:
eda.plot_boxplots(df_imputed, variables_numericas)

### Horas de estudio en casa

In [22]:
# Llenamos los valores nulos con la media
df_imputed['horas_semana_estudio_casa'] = df_imputed['horas_semana_estudio_casa'].fillna(df_imputed['horas_semana_estudio_casa'].mean())

### Número de hermanos

In [23]:
# Llenamos los valores nulos con la mediana
df_imputed['total_hermanos'] = df_imputed['total_hermanos'].fillna(df_imputed['total_hermanos'].median())

## Manejo de valores atípicos

In [24]:
cols_to_search = [col for col in df.columns if col not in ['ID','sede', 'nivel', 'grado', 'orden_grado', 'año_ingreso', 'género', 'documento_identificación', 'primer_apellido', 'segundo_apellido', 'nombres', 'fecha_nacimiento', 'dirección', 'valoración_emocional']]
cols_to_search

['antigüedad',
 'país_origen',
 'estrato',
 'tipo_vivienda',
 'zona_vivienda',
 'horas_semana_estudio_casa',
 'interés_estudios_superiores',
 'medio_transporte',
 'apoyo_familiar',
 'total_hermanos',
 'familia',
 'actividades_extracurriculares',
 'enfermedades',
 'proyección_vocacional',
 'participación_clase',
 'nee',
 'nivel_motivación']

In [25]:
variables_numericas = ['horas_semana_estudio_casa', 'total_hermanos']

In [26]:
eda.detect_outliers(df_imputed, variables_numericas)

{'horas_semana_estudio_casa': 12      7.0
 58     30.0
 62      8.0
 76      9.0
 78      6.0
 186    30.0
 196    68.0
 225    20.0
 229    30.0
 243     6.0
 269     6.0
 272     7.0
 274     8.0
 284     7.0
 293    10.0
 361    10.0
 Name: horas_semana_estudio_casa, dtype: float64,
 'total_hermanos': 1      0.0
 2      0.0
 5      3.0
 6      0.0
 7      4.0
       ... 
 356    2.0
 362    0.0
 363    0.0
 364    0.0
 365    0.0
 Name: total_hermanos, Length: 145, dtype: float64}

### Horas de estudio en casa

In [27]:
eda.plot_boxplots(df_imputed, ['horas_semana_estudio_casa'])

### Número de hermanos

In [28]:
eda.plot_boxplots(df_imputed, ['total_hermanos'])

## Manejo de atípicos categóricos

In [29]:
variables_categoricas = list(set(cols_to_search) - set(variables_numericas))
variables_categoricas

['nivel_motivación',
 'proyección_vocacional',
 'tipo_vivienda',
 'participación_clase',
 'país_origen',
 'estrato',
 'actividades_extracurriculares',
 'medio_transporte',
 'familia',
 'zona_vivienda',
 'apoyo_familiar',
 'interés_estudios_superiores',
 'enfermedades',
 'antigüedad',
 'nee']

In [30]:
rare_categories = eda.detect_rare_categories(df_imputed, variables_categoricas)
rare_categories

{'país_origen': ['Argentina', 'España', 'República dominicana'],
 'actividades_extracurriculares': ['Artes, Idiomas',
  'Artes, Deporte, Idiomas',
  'Tecnología / Diseño'],
 'medio_transporte': ['Bicicleta']}

In [31]:
eda.plot_barplots("Categorías Atípicas", df_imputed, list(rare_categories.keys()))

In [32]:
eda.plot_null_heatmap(df_imputed)

In [33]:
# Guardamos el dataframe limpio
df_imputed.to_csv('../../data/raw/Estudiantes_imputed.csv', index=False)